<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
################################################################################
#Author: Arif H. Zaidy                                                         #
#Date: March 07, 2025                                                          #
#Course: CPSC 5440                                                             #
#Topic: Assignment 2                                                           #
#Description:                                                                  #
#This program performs hyperparameter tuning on a neural network using         #
#the CIFAR-100 dataset. It loads the dataset from Google Drive, defines        #
#a search space for hyperparameters, and trains models using different         #
#configurations. The best-performing model is selected based on validation     #
#accuracy, and results are visualized using a line plot. Finally, the script   #
saves the results and plots to Google Drive for further analysis.             #
################################################################################

# Including Python libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from itertools import product
from tqdm import tqdm
from google.colab import drive

# Connecting to Google Drive
drive.mount("/content/drive")

# Load CIFAR-100 dataset
def load_data():
    with open('/content/drive/My Drive/train', 'rb') as file:
        train_dict = pickle.load(file, encoding='bytes')
    with open('/content/drive/My Drive/test', 'rb') as file:
        test_dict = pickle.load(file, encoding='bytes')

    X_train = train_dict[b'data']
    y_train = train_dict[b'coarse_labels']
    X_test = test_dict[b'data']
    y_test = test_dict[b'coarse_labels']

    enc = OneHotEncoder(sparse_output=False, categories='auto')
    y_train = enc.fit_transform(np.array(y_train).reshape(-1, 1))
    y_test = enc.transform(np.array(y_test).reshape(-1, 1))

    X_train = torch.tensor(X_train / 255.0, dtype=torch.float32).reshape(-1, 3072)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_test = torch.tensor(X_test / 255.0, dtype=torch.float32).reshape(-1, 3072)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    return X_train, y_train, X_test, y_test

# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size=3072, hidden_size=240, num_classes=100, activation_fn=nn.ReLU):
        super(SimpleNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            activation_fn(),
            nn.Linear(hidden_size, hidden_size),
            activation_fn(),
            nn.Linear(hidden_size, num_classes)
        )
        self.history = {'epoch': [], 'accuracy': []}  # Store history in the model

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = self.fc_layers(x)
        return x

# Train function
def train_model(X_train, y_train, X_test, y_test, params):
    model = SimpleNN(input_size=3072, hidden_size=params['units'], num_classes=100,
                      activation_fn=nn.ReLU if params['hidden_activations'] == 'relu' else nn.Sigmoid)

    criterion = nn.CrossEntropyLoss() if params['loss'] == 'categorical_crossentropy' else nn.MSELoss()
    optimizer = optim.Adam(model.parameters()) if params['optimizer'] == 'adam' else optim.Adagrad(model.parameters())

    train_loader = data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=params['batch_size'], shuffle=True)
    test_loader = data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=params['batch_size'], shuffle=False)

    best_acc = 0.0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(5):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/5", leave=True)

        for batch_X, batch_y in loop:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)

            if isinstance(criterion, nn.CrossEntropyLoss):
                batch_y = torch.argmax(batch_y, dim=1).long()

            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                _, predicted = torch.max(outputs, 1)
                _, labels = torch.max(batch_y, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        accuracy = correct / total
        print(f"Epoch {epoch+1}/5: Accuracy = {accuracy * 100:.2f}%")

        model.history['epoch'].append(epoch)
        model.history['accuracy'].append(accuracy)

        if accuracy > best_acc:
            best_acc = accuracy
            best_params = params

    return model, model.history, best_acc, best_params

# Define parameter grid
param_grid = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'loss': ['categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [128, 256]
}

param_combinations = list(product(*param_grid.values()))

# Load data
X_train, y_train, X_test, y_test = load_data()

df_results = []
best_overall_acc = 0.0
best_hyperparams = None
for param_values in param_combinations:
    params = dict(zip(param_grid.keys(), param_values))
    _, history, acc, best_params = train_model(X_train, y_train, X_test, y_test, params)
    print(best_params)
    df_results.append({**params, 'accuracy': acc})
    if acc > best_overall_acc:
        best_overall_acc = acc
        best_hyperparams = best_params

df_results = pd.DataFrame(df_results)
print("Best Hyperparameter Combination:", best_hyperparams)
print("Best Overall Accuracy:", best_overall_acc)

plt.figure(figsize=(10, 5))
sns.lineplot(data=df_results, x=df_results.index, y='accuracy', hue='hidden_activations')
plt.xlabel('Training Iteration')
plt.ylabel('Validation Accuracy')
plt.title('Hyperparameter Tuning Results')
plt.grid()
plt.savefig("/content/drive/My Drive/hyperparameter_tuning_plot.png", dpi=300)
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch 1/5: 100%|██████████| 391/391 [00:02<00:00, 133.14it/s, loss=2.46]


Epoch 1/5: Accuracy = 20.30%


Epoch 2/5: 100%|██████████| 391/391 [00:02<00:00, 131.84it/s, loss=2.28]


Epoch 2/5: Accuracy = 25.50%


...

Epoch 4/5: Accuracy = 21.56%


Epoch 5/5: 100%|██████████| 196/196 [00:02<00:00, 66.83it/s, loss=2.23]


Epoch 5/5: Accuracy = 21.23%
Best Hyperparameter Combination: {'units': 240, 'hidden_activations': 'relu', 'loss': 'categorical_crossentropy', 'optimizer': 'adam', 'batch_size': 128}
Best Overall Accuracy: 0.3049


TypeError: Data source must be a DataFrame or Mapping, not <class 'list'>.

<Figure size 1000x500 with 0 Axes>